In [31]:
from __future__ import annotations
import sys
import unittest
from pathlib import Path

from ivonet.calc import base_x_to_10
from ivonet.files import read_data
from ivonet.iter import ints
from ivonet.str import chunk_string
from ivonet.files import read_int_matrix
from ivonet.grid import neighbors_defined_grid
from ivonet.iter import ints
from ivonet.search import astar

In [10]:
def get_rotations(x, y, z):
    return [
        (x, y, z),
        (-y, x, z),
        (-x, -y, z),
        (y, -x, z),
    ]


def get_z_orientations(x, y, z):
    return [
        (x, y, z),
        (x, z, -y),
        (x, -y, -z),
        (x, -z, y),
        (-z, y, x),
        (z, y, -x),
    ]


def get_orientations(x, y, z):
    for xi, yi, zi in get_z_orientations(x, y, z):
        yield from get_rotations(xi, yi, zi)
        

In [23]:
a = list(get_orientations(-1,-1,1))
a

[(-1, -1, 1),
 (1, -1, 1),
 (1, 1, 1),
 (-1, 1, 1),
 (-1, 1, 1),
 (-1, -1, 1),
 (1, -1, 1),
 (1, 1, 1),
 (-1, 1, -1),
 (-1, -1, -1),
 (1, -1, -1),
 (1, 1, -1),
 (-1, -1, -1),
 (1, -1, -1),
 (1, 1, -1),
 (-1, 1, -1),
 (-1, -1, -1),
 (1, -1, -1),
 (1, 1, -1),
 (-1, 1, -1),
 (1, -1, 1),
 (1, 1, 1),
 (-1, 1, 1),
 (-1, -1, 1)]

In [24]:
len(a)

24

In [16]:
def roll(v):  # Hand rule
    return v[0], v[2], -v[1]


def turn(v):
    return -v[1], v[0], v[2]


def sequence(v):
    for cycle in range(2):
        for step in range(3):  # Yield RTTT 3 times
            v = roll(v)
            yield v  # Yield R
            for i in range(3):  # Yield TTT
                v = turn(v)
                yield v
        v = roll(turn(roll(v)))  # Do RTR


In [26]:
b = list(sequence((-1,-1,1)))
b

[(-1, 1, 1),
 (-1, -1, 1),
 (1, -1, 1),
 (1, 1, 1),
 (1, 1, -1),
 (-1, 1, -1),
 (-1, -1, -1),
 (1, -1, -1),
 (1, -1, 1),
 (1, 1, 1),
 (-1, 1, 1),
 (-1, -1, 1),
 (-1, 1, -1),
 (-1, -1, -1),
 (1, -1, -1),
 (1, 1, -1),
 (1, -1, -1),
 (1, 1, -1),
 (-1, 1, -1),
 (-1, -1, -1),
 (-1, -1, 1),
 (1, -1, 1),
 (1, 1, 1),
 (-1, 1, 1)]

In [28]:
len(b)

24

In [33]:
class Scanner:

    def __init__(self, id: int, beacons: list[tuple[int, int, int]]) -> None:
        self.id = id
        self.orig = beacons.copy()
        self.current = beacons.copy()
        self.length = len(beacons)
        self.fixed_beacons = None
        self.locked = False

    def roll(self):
        ret = []
        for c in self.current:
            ret.append(roll(c))
        self.current = ret

    def turn(self):
        ret = []
        for c in self.current:
            ret.append(turn(c))
        self.current = ret

    def all(self):
        """https://stackoverflow.com/questions/16452383/how-to-get-all-24-rotations-of-a-3-dimensional-array
        """
        # if self.locked:
        #     yield self
        for _ in range(2):
            for action in "RTTTRTTTRTTT":
                if action == "R":
                    self.roll()
                    yield self.current
                else:
                    self.turn()
                    yield self.current
            # reorient to do the other 12 positions
            self.roll()
            self.turn()
            self.roll()

    def reset(self):
        self.current = self.orig.copy()

    def matches(self, scanner: Scanner) -> bool:
        ...

    def __str__(self) -> str:
        s = "\n ".join([str(x).replace(" ", "") for x in self.current])
        return f"[{s}]"

    def __repr__(self) -> str:
        return f"Scanner[{self.id}]"

In [35]:
s0 = [(-618, -824, -621),
              (-537, -823, -458),
              (-447, -329, 318),
              (404, -588, -901),
              (544, -627, -890),
              (528, -643, 409),
              (-661, -816, -575),
              (390, -675, -793),
              (423, -701, 434),
              (-345, -311, 381),
              (459, -707, 401),
              (-485, -357, 347)]
s1 = [(686, 422, 578),
              (605, 423, 415),
              (515, 917, -361),
              (-336, 658, 858),
              (-476, 619, 847),
              (-460, 603, -452),
              (729, 430, 532),
              (-322, 571, 750),
              (-355, 545, -477),
              (413, 935, -424),
              (-391, 539, -444),
              (553, 889, -390)]
sc0 = Scanner(0, s0)
sc1 = Scanner(1, s1)
sc